**Code for Paper "The reconstruction of flows from spatiotemporal data by autoencoders"**

Facundo Fainstein (1,2), Josefina Catoni (1), Coen Elemans (3) and Gabriel B. Mindlin (1,2,4)* 

(1) Universidad de Buenos Aires, Facultad de Ciencias Exactas y Naturales, Departamento de Física, Ciudad Universitaria, 1428 Buenos Aires, Argentina.

(2) CONICET - Universidad de Buenos Aires, Instituto de Física Interdisciplinaria y Aplicada (INFINA), Ciudad Universitaria, 1428 Buenos Aires, Argentina.

(3) Department of Biology, University of Southern Denmark, 5230 Odense M, Denmark.

(4) Universidad Rey Juan Carlos, Departamento de Matemática Aplicada, Madrid, Spain. 

*Gabriel B. Mindlin (corresponding author)
Email: gabo@df.uba.ar


**Train autoencoder** 

Fit 100 autoencoders. The architecture is: 9900-64-32-16-2-16-32-64-9900. Save the results in .txt files. 

In [ ]:
# Python libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
from numpy import loadtxt
from skimage import io
import os
from pathlib import Path

#Keras
import keras
from keras import layers
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras import backend as K

#Uncomment the following lines if run in google colab 

# from google.colab import drive
#Mount drive to load images
#drive.mount('/content/gdrive', force_remount=True)  

**Load data**

In [ ]:
#Load data

#Movie folder
root_dir = f'/media/.../' 
#Save file names in list
lista_files=[]
lista_files=glob.glob(root_dir+'*.jpg')
lista_files.sort()

#Load data
data = []
for file in lista_files:
  img = io.imread(file, as_gray=True)
  data.append(img)  
data = np.array(data)
print(data.shape)

#Define the pixels of each images to analyze
x_der = data[:,120:230,180:270].astype('float32')

#Compute the temporal mean value of each pixel
xmean_der=np.mean(x_der,axis=0) 

#Substract the mean 
x_der -= xmean_der
print(type(x_der))
print(x_der.shape)

**Define a function that fits the network and saves the results**

In [ ]:
#Function that creates and trains the network, and saves the results
def train_save(X, model_path):
    # X = data
    # X = [time, rows*columns pixels]
    # model_path = path to folder in which results will be saved
    
    #Separate train and test data sets
    X_train, X_test = X[0:300], X[300:]
    
    #Define the network
    numero_pixeles = X.shape[1]
    input_img = keras.Input(shape=(numero_pixeles,))
    encoded = layers.Dense(64, activation='relu')(input_img)
    encoded = layers.Dense(32, activation='relu')(encoded)
    encoded = layers.Dense(16, activation='relu')(encoded)
    encoded = layers.Dense(2, activation='linear')(encoded)
    decoded = layers.Dense(16, activation='relu')(encoded)
    decoded = layers.Dense(32, activation='relu')(decoded)
    decoded = layers.Dense(64, activation='relu')(decoded)
    decoded = layers.Dense(numero_pixeles, activation='linear')(decoded)#Create the autoencoder
    autoencoder = keras.Model(input_img, decoded)

    #Compile the model
    autoencoder.compile(loss='mse', metrics=['mean_absolute_error'], optimizer='adam')

    # training the model and saving metrics in history
    history = autoencoder.fit(X_train, X_train,
                batch_size=16, epochs=200,
                validation_data=(X_test, X_test),
                verbose=0);

    #Get results from trained net
    train_mse = np.array(history.history['loss'])  #MSE for the train data set
    val_mse = np.array(history.history['val_loss']) #MSE for the test data set
    get_latent_layer_output = K.function([autoencoder.layers[0].input],[autoencoder.layers[4].output])
    layer_output_train = get_latent_layer_output ([X_train])[:][0]  #Latent space projection for the training set
    prediction_test = autoencoder.predict(X_test)  #Output of the test set
    layer_output_test = get_latent_layer_output ([X_test])[:][0] #Latent space projection for the test set

    #Save results 
    np.savetxt(model_path+'/train_mse.txt', train_mse)
    np.savetxt(model_path+'/val_mse.txt', val_mse)
    np.savetxt(model_path+'/layer_output_train.txt', layer_output_train)
    np.savetxt(model_path+'/layer_output_test.txt', layer_output_test)
#     np.savetxt(model_path+'/recostruction_test.txt', prediction_test) #Uncomment if needed

    return train_mse,val_mse,layer_output_train,layer_output_test

**Fit 100 networks**

In [ ]:
#Define data 
X = x_der.reshape(x_der.shape[0], x_der.shape[1]*x_der.shape[2])
#Compute velocity of the graphical projection
d_frames_test = np.sqrt(np.sum(np.diff(X[300:],axis=0)**2,axis=1))

#Number of trainings
N=100
#Path to save folder
path = f'/home/.../'

for k in range(N):
    
    print('iteracion ' + str(k) + ' de '+str(N))
    #Create name to save training results
    directory= 'it{}'.format(k)
    #Path to saving folder, create directory if necessary
    model_path = os.path.join(path, directory)
    Path(model_path).mkdir(parents=True, exist_ok=True)

    #Train and save
    (train_mse,val_mse,layer_output_train,
    layer_output_test) = train_save(X,model_path=model_path)
  
    #Compute velocity in latent space 
    d_ls_test = np.sqrt( np.diff(layer_output_test[:,0])**2 + 
                              np.diff(layer_output_test[:, 1])**2 ) 

    #Plot result
    plt.figure(figsize=(15,3))
    plt.subplot(1,5,1)
    plt.title(directory)
    plt.plot(layer_output_train[:,0])
    plt.plot(layer_output_train[:,1])
    plt.subplot(1,5,2)
    plt.plot(layer_output_test[:,0])
    plt.plot(layer_output_test[:,1])      
    plt.subplot(1,5,3)
    plt.plot(layer_output_test[:,0],layer_output_test[:,1],'.')
    plt.subplot(1,5,4)
    plt.plot(train_mse, label="train")
    plt.plot(val_mse, label="test")
    plt.ylim([min(val_mse)-min(val_mse)*0.2, val_mse[25]+val_mse[25]*0.2])
    plt.subplot(1, 5, 5)
    plt.plot(d_frames_test, c='k', label="d frames")
    plt.gca().twinx().plot(d_ls_test, alpha=0.7, label="d l-space")
    plt.legend()
    plt.show()